# 1. 집 고를 때

In [1]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [3]:
system_template="""
너는 사용자의 전세사기 예방을 돕는 챗봇이야. 너는 여러 챗봇 중 "집을 고르는 단계" 시점의 사용자를 담당해
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [4]:
human_template="""
깡통주택이란? :
깡통주택이란, 임대인의 집이 경매로 넘어갔을 때, 내 보증금이 떼이게 되는 집을 통틀어 가리켜요. 위 그림처럼 매매가격의 대다수를 세입자의 보증금과 빚으로 채우고 있는 집이어서, 사실상 임대인의 몫은 거의 없는 집들이 흔히 깡통주택이에요.
보통 깡통주택을 전세로 내놓는 임대인들은 ‘갭투기’를 통해 해당 주택을 사들였을 거예요. 이때의 갭(차이)은 ‘매매가-전세가’를 말해요. 그 ‘갭’마저도 은행에서 대출을 받고요. 이렇게 되면, 임대인이 스스로 자유롭게 융통할 수 있는 돈은 거의 없죠. 보증금도, 은행 대출금도 전부 언젠가 타인에게 돌려줘야 하는 돈이니까요. 그렇기 때문에 만약 집이 경매에 넘어가기라도 하면, 세입자는 보증금을 떼이게 되고요. 경매에 넘어가지 않더라도, 계약이 끝나고 이사를 나갈 때 제때 보증금을 돌려주지 못할 수도 있어요. 임대인이 제 돈 주고 산 집이 아니니, 돌려줄 돈이 없는 경우에 특히 더 그렇죠. 이런 경우, 다음 세입자가 구해지지 않아서 보증금을 줄 수 없다고 해요. 또는, 집 가격이 하락해 집값이 전세보증금보다 적어졌을 때에도 이사를 가려는 세입자에게 보증금을 돌려줄 수 없게 되는 일이 발생할 수도 있어요.
일반적으로 깡통주택은 보증금 + 대출금의 총합이 집값의 80%를 넘는 집을 의미해요. 즉, 임대인이 자기 돈을 얼마나 들여서 집을 갖고 있느냐가 중요해요. 깡통주택의 임대인은 불상사가 생겼을 때 빚 상환을 회피해버릴 수 있기 때문이죠. 그렇기에 깡통주택은 마치 시한폭탄 같은 존재라고 할 수 있어요.
깡통주택에 전세로 들어간 순간부터 보증금을 돌려받지 못할 가능성이 생겨요. 예를 들어, 주택A,B 모두 경매에 넘어갔을 때, 주택A에서는 보증금을 전부 돌려받지 못할 수 있어요. 경매에 넘어가지 않더라도, 집값이 떨어지거나 임대인의 경제적 사정이 어려워지면 보증금을 돌려주지 못하게 될 수도 있어요. 애초에 임대인의 돈은 거의 없는 깡통주택이기 때문에, 임대인이 돈이 없다며 보증금을 돌려주지 않을 수도 있죠. 그러니 반드시 등기부등본 서류를 통해 이 집에 빚이 얼마나 많은지 확인해야 해요.

사례 : 한동안 이슈가 되었던 ‘대구 깡통전세사기 사건’을 아시나요? 올해 초, 임대인 장 모 씨가 세입자 50여 명에게 약 68억 원의 보증금을 돌려주지 않았다는 사실이 드러나 충격을 안겨줬어요. 장 씨는 대구에서 다가구주택 13채를 갭투기로 사들인 뒤, 세입자에게 집을 빌려줬어요. 이후 보증금을 돌려받아야 하는 세입자들에게 돈을 돌려주지 않았어요. 그 결과, 장 씨는 2019년과 2020년에 각각 징역 3년, 4년형을 선고 받았어요. 이 사건이 바로 깡통주택 전세사기예요. 깡통전세 사기를 당하면, 임대인이 처벌을 받게 된다고 해도, 세입자들 모두가 보증금 전부를 돌려받을 수 있는 건 아니라고 해요. 그러니 더욱 유의해야 해요!

등기부등본을 확인해야하는 이유 :
보증금을 지키기 위해서는, 어떤 집을 선택할지 고를 때부터 그 집의 등기부등본을 꼭 확인해야 해요. 아무리 채광이 좋은 집이라고 해도, 빚이 너무 많아서 나중에 내 보증금을 제때 돌려주지 못한다면, 분명 위험한 집이니 피해야겠죠! 반드시 피해야 하는 집에 대한 단서가 바로 등기부등본에 있답니다.
등기부등본에서는 ①임대인이 누구인지, ②빚이 얼마나 많은지(근저당권 등), ③혹시 누군가의 보증금을 떼어먹은 전적은 없는지(임차권) 등의 정보를 확인할 수 있어요.그리고 등기부등본은 계약하기 전에도, 계약 이후 잔금을 치르기 직전에도 반드시 확인해야 해요. 왜냐하면 내가 입주하기 전에 임대인이 나와 계약한 집을 담보로 빚을 지거나, 아니면 매매를 해버릴 수도 있기 때문이에요.
약, 아파트나 오피스텔 같은 집합건물을 알아보고 있다면, ‘토지’와 ‘건물’을 하나의 등기부등본에서 함께 확인할 수 있어요. 하지만 다가구주택, 다세대주택, 단독주택 등 집합 건물이 아닌 집을 알아보는 중이라면, 토지에 대한 등기부등본도 함께 확인해봐야 해요. 건물의 등기부등본이 깨끗해도, 토지 등기부등본이 복잡하게 꼬여 있는 집도 있답니다. 등기부등본은 임대인이 아니더라도 누구나 대법원 인터넷 등기소*에서 발급 및 열람할 수 있으므로 확인을 주저하지 마세요. 열람은 700원! 700원으로 나의 보증금을 안전하게 지켜요!
※단, 임차권자가 임차권등기를 아직 마치지 못한 경우에는 등기부등본에 적히지 않을 수 있습니다. 이 경우, 전입세대 열람내역, 확정일자 부여 현황 등으로 확인해야 합니다.
대법원 인터넷 등기소 링크 : http://www.iros.go.kr/PMainJ.jsp

등기부등본 보는 법 :
등기부등본은 크게 ①표제부 ②갑구 ③을구, 이 세 가지 영역으로 구성되어 있어요. 
표제부는 부동산에 대한 일반 현황을 확인할 수 있고, 갑구에서는 소유권에 대한 권리를 확인 할 수 있고, 을구에서는 소유권 외 권리를 확인할 수 있어요.
등기부등본의 ‘갑구’에는 소유권에 관한 내용이 적혀 있어요. 말 그대로 이 집을 소유하고 있는 사람이 누구인지에 대한 정보가 있죠. 그리고 그러한 소유권을 제한하고 있는 문제 상황은 없는지 살펴볼 수 있어요. 압류 / 가압류 / 가등기 등 주택에 대한 소유권을 제한할 수 있는 조건이 기록되어 있죠. 예를 들어, 소유주 이름이 ‘김ㅇㅇ’이라고 해도, 세무서가 ‘압류’를 걸어뒀다면, 그 집은 위험한 상태인 거예요. 압류 / 가압류 / 가등기 등 소유권을 침해할 수 있는 요소가 등기부등본에 적혀 있다면 전세계약의 효력 자체가 사라질 수도 있으니까 전세계약을 체결하지 않는 것이 좋아요.
등기부등본의 ‘을구’에는 이 집에 얽힌 돈 얘기들이 가득 적혀 있어요. ①이미 이 집에 전세권을 가진 사람이 있거나 ②임대인이 집을 담보로 많은 빚을 졌거나, 하는 것들을 을구에서 확인할 수 있어요.
등기부 등본을 보는 방법에 대한 자세한 내용 링크 : https://www.khug.or.kr/jeonse/web/s03/s030105.jsp

규칙 :
1. 질문이 특정 사기 유형에 해당하지 않는다면 답변에는 깡통주택의 정의가 포함되어야 한다.
2. 등기부등본에 대해서 말할 때는 표제부, 갑구, 을구를 최대한 자세하게 설명해야한다.

위 내용과 규칙을 참고해서 다음 Question에 500단어 정도로 답변해.
Question :  {text}
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [5]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [6]:
# prompt template가 3000 이며 사용자의 질문을 고려해서 600으로 산정
# llm = ChatOpenAI(temperature=0, max_tokens=1000)
llm = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0, max_tokens=600)
select_home_chain = LLMChain(
    llm=llm,
    prompt=chat_prompt
)

In [7]:
select_home_chain.run("전세집을 구하려고 하는데 사기예방을 위해 꼭 알아야하는 점이 있을까?")

전세집을 구하려는 경우, 깡통주택 사기와 같은 부동산 사기에 대한 경각심을 가져야 합니다. 이를 위해서는 몇 가지 중요한 점을 꼭 알아야 합니다.

먼저, 깡통주택이란 무엇인지 알아야 합니다. 깡통주택은 임대인이 갭투기로 집을 사들인 뒤, 세입자의 보증금과 빚으로 집값을 채우는 집을 말합니다. 이러한 집은 보증금과 대출금의 총합이 집값의 80%를 넘는 경우가 많아, 세입자가 보증금을 돌려받지 못할 가능성이 높습니다. 따라서 전세집을 구할 때는 반드시 등기부등본을 확인해야 합니다. 등기부등본에서는 집의 소유자, 빚 상황, 임차권 등의 정보를 확인할 수 있습니다. 이를 통해 깡통주택인지 아닌지를 판단할 수 있습니다.

또한, 전세계약을 체결할 때는 계약서를 꼼꼼히 확인해야 합니다. 계약서에는 보증금, 월세, 계약 기간 등의 내용이 포함되어 있습니다. 이를 잘 읽어보고, 이해하지 못하는 부분이 있다면 반드시 질문해야 합니다. 또한, 계약서에는 임대인과 세입자의 권리와 의무가 명시되어 있으므로, 이를 잘 숙지해야 합니다.

또한, 전세집을 구할 때는 중개업자를 통해 매물을 찾는 경우가 많습니다. 이때는 중개업자의 신뢰성을 확인해야 합니다. 중개업자는 전문적인 지식과 경험을 가지고 있으므로, 전세집 구매에 대한 조언을 제공해줄 수 있습니다. 하

'전세집을 구하려는 경우, 깡통주택 사기와 같은 부동산 사기에 대한 경각심을 가져야 합니다. 이를 위해서는 몇 가지 중요한 점을 꼭 알아야 합니다.\n\n먼저, 깡통주택이란 무엇인지 알아야 합니다. 깡통주택은 임대인이 갭투기로 집을 사들인 뒤, 세입자의 보증금과 빚으로 집값을 채우는 집을 말합니다. 이러한 집은 보증금과 대출금의 총합이 집값의 80%를 넘는 경우가 많아, 세입자가 보증금을 돌려받지 못할 가능성이 높습니다. 따라서 전세집을 구할 때는 반드시 등기부등본을 확인해야 합니다. 등기부등본에서는 집의 소유자, 빚 상황, 임차권 등의 정보를 확인할 수 있습니다. 이를 통해 깡통주택인지 아닌지를 판단할 수 있습니다.\n\n또한, 전세계약을 체결할 때는 계약서를 꼼꼼히 확인해야 합니다. 계약서에는 보증금, 월세, 계약 기간 등의 내용이 포함되어 있습니다. 이를 잘 읽어보고, 이해하지 못하는 부분이 있다면 반드시 질문해야 합니다. 또한, 계약서에는 임대인과 세입자의 권리와 의무가 명시되어 있으므로, 이를 잘 숙지해야 합니다.\n\n또한, 전세집을 구할 때는 중개업자를 통해 매물을 찾는 경우가 많습니다. 이때는 중개업자의 신뢰성을 확인해야 합니다. 중개업자는 전문적인 지식과 경험을 가지고 있으므로, 전세집 구매에 대한 조언을 제공해줄 수 있습니다. 하'

# 2. 임대인 확인할 때

In [8]:
system_template="""
너는 사용자의 전세사기 예방을 돕는 챗봇이야. 너는 여러 챗봇 중 "임대인 확인할 때" 시점의 사용자를 담당해
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [9]:
human_template="""
임대인 확인할 때 생길 수 있는 전세사기 유형에는 2가지가 있습니다. 아래는 각 전세사기의 유형과 각 유형별 사례와 예방,대처법이 있습니다.
1. 가짜 임대인과의 계약
사례 : 2019년, 인천 중구에서 있었던 일이에요. A씨는 타인의 오피스텔을 자신의 소유라고 거짓말하면서 등기부등본의 소유자 항목을 위조하여 사기를 쳤어요. A씨가 바로 ‘가짜 임대인’이었던 것이죠. 피해자들의 대부분은 아직 임대차 경험이 많지 않은 사회 초년생이었죠. 금방 들통날 얄팍한 거짓말었지만, 그 피해가 생각보다 어마어마했어요. 무려 약 24억원에 달했죠.
이처럼 임대인이 아닌 사람이 임대인인 척 명의를 도용하는 경우가 심심치 않게 발생해요. ‘가짜 임대인’이 건물을 임대하면서 알아낸 ‘실제 임대인’의 인적사항을 이용하여 자신이 건물주인 척 거짓된 전세계약을 한 후 세입자의 보증금을 가로채는 거죠. 혹은 임대인의 위임장 또는 증명서류를 위조해 대리인 행세를 하면서 세입자의 보증금을 가로채기도 해요.
이런 불상사를 막기 위해서는 임대인이라고 주장하는 사람이 정말 임대인이 맞는지를 확인하는 일이 중요해요. 신분증에 쓰여 있는 임대인의 인적사항과 등기부등본 상의 임대인의 정보가 일치하는지 확인해야 하겠죠. 이때 등기부등본은 본인이 직접 발급받아 확인해보기를 추천해요. 등기부등본을 위조하는 경우도 있다고 하니까요.
임대인을 대신해서 대리인이 왔을 때 대처법 :
- 임대인의 신분증을 대체할 수 있는 서류들 즉, 인감도장이 찍힌 위임장과 그 인감도장을 증명할 인감증명서를 확인해야만 해요. 위임장과 인감증명서를 대조해 두 서류에 찍힌 임대인의 인감이 동일한지 체크하는 것이죠.
- 임대인과의 유선 통화를 통해 계약 내용을 다시 한 번 확인하기를 추천해요.
- 전세보증금을 반드시 임대인 명의의 통장에 지급해야 해요. 만일 임대인의 요청에 의해 대리인 계좌로 입금해야 하는 경우라면, 특약사항에 임대인의 요청에 따라 대리인 계좌로 입금하게 되었음을 명시해야만 합니다.

2. 신탁회사의 동의 없는 계약
사례 : 2019년, 마산에 사는 임대인 A씨는 2019년 1월에 한 오피스텔을 신탁회사에 넘기고 그 증서를 담보로 거액의 대출을 받았어요. 그럼 임대인 A씨에게는 신탁회사의 동의 없이는 집을 임대할 권한이 없게 되는데, A씨는 이를 속이고 15가구의 세입자들과 계약을 강행해버렸어요. 보증금을 가로챈 거죠. 신탁회사 동의 없이 계약했으니, 계약 자체가 무효가 됐어요. 세입자들은 하루아침에 쫓겨나게 됐고요. 그렇게 세입자들은 총 5억 원 규모의 신탁 부동산 사기 피해를 입게 됐어요.
신탁이란? - 신탁이란, 임대인이 주택을 비롯한 자신의 부동산을 전문가에게 맡기는 걸 의미해요. 임대인은 주택을 담보로 대출을 받을 수도 있고요, 혹은 다른 수익을 얻기 위해서도 신탁을 해요.
신탁등기와 일반등기를 구분하는 법 - 등기부등본 서류에는 표제부, 갑구, 을구가 있어요. 이 중 ‘갑구’를 보면, ‘신탁’ 여부를 바로 알아볼 수 있어요. 임대인이 신탁회사에게 신탁 계약을 맺었다는 것이 ‘갑구’에 적혀 있다면, 진짜 소유권은 임대인에게 없어요. 보통은 임대인이 곧 집의 소유주인지 확인한 뒤 계약서에 서명을 하는데, 만약 신탁등기라면 이렇게 계약해선 안 돼요.
신탁등기 전세사기 예방법 - 잘못된 계약을 하지 않으려면, 등기부등본을 통해 부동산이 신탁되었는지 여부를 확인해야만 해요. 만일 등기부등본을 확인했는데 신탁등기가 되어 있다면 ‘신탁원부’라는 서류를 추가적으로 확인해야만 하고요. 신탁원부란, 위탁자와 수탁자 그리고 수익자와 신탁관리인의 성명 및 주소, 신탁의 목적, 신탁재산의 관리방법, 신탁 종료의 사유 등을 포함한 서류예요. 따라서 신탁원부를 확인하면 누구에게 이 부동산을 임대해줄 권한이 있는지, 해당 부동산을 담보로 받은 대출이 있는지 등을 알 수 있어요. 참고로 이 신탁원부는 인터넷 으로는 발급받을 수 없기 때문에 등기소에서 직접 발급받아야만 해요. 그리고 한 가지 말씀드리자면, 신탁원부를 통해 권리관계를 파악하는 일이 개인에게는 다소 생소하고 어려울 수 있기 때문에 전문가의 도움을 받기를 추천해요.

규칙 :
1. 질문이 특정 사기 유형에 해당하지 않는다면 먼저 전세사기 유형 2가지에 대해 답변에서 설명한다.
2. 해당하는 유형별 대처 방법을 반드시 답변에 포함한다.

위 내용과 규칙을 참고해서 다음 Question에 1500단어 정도로 답변해.
Question :  {text}
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [10]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [11]:
# prompt template가 1817이고, 사용자의 질문을 고려해서 1800으로 산정
# llm = ChatOpenAI(temperature=0, max_tokens=1800)
llm = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0, max_tokens=1800)
check_lessor_chain = LLMChain(
    llm=llm,
    prompt=chat_prompt
)

In [12]:
check_lessor_chain.run("전세집을 구하려고 하는데 요즘 가짜임대인을 이용한 사기가 많다고 하더라구, 임대인을 확인해보니 본인이 맞아, 더 조심해야할 부분은 없을까?")

가짜 임대인을 이용한 전세사기는 매년 많은 피해자들을 만들고 있습니다. 따라서 임대인을 확인하는 것은 전세집을 구할 때 가장 중요한 일 중 하나입니다. 하지만, 임대인이 본인이 맞다고 해서 모든 것이 끝나는 것은 아닙니다. 여전히 조심해야할 부분이 있습니다.

먼저, 임대인이 본인이 맞는지 확인하는 것은 등기부등본을 통해 확인할 수 있습니다. 등기부등본은 부동산의 소유자와 관련된 정보를 담고 있는 공적 서류입니다. 등기부등본을 확인하면 임대인의 인적사항과 부동산의 소유자 정보를 확인할 수 있습니다. 하지만, 등기부등본을 위조하는 경우도 있기 때문에 등기부등본을 확인할 때는 본인이 직접 발급받아 확인하는 것이 좋습니다.

또한, 가짜 임대인이 등장하는 경우 대리인이 나타날 수도 있습니다. 대리인이 나타난다면, 대리인의 신분증을 확인하고 대리인이 임대인의 위임을 받았는지 확인해야 합니다. 대리인이 임대인의 위임을 받았다면, 위임장과 인감증명서를 대조해 두 서류에 찍힌 임대인의 인감이 동일한지 체크하는 것이 중요합니다. 또한, 임대인과의 유선 통화를 통해 계약 내용을 다시 한 번 확인하는 것도 좋은 방법입니다.

마지막으로, 전세보증금을 지불할 때는 반드시 임대인 명의의 통장에 지불해야 합니다. 만약 임대인의 요청에 의해 대리인 계좌로 입금해야 하는 경우라면, 특약사항에 임대인의 요청에 따라 대리인 계좌로 입금하게 되었음을 명시해야 합니다. 이를 통해 보증금을 가로채는 사기를 방지할 수 있습니다.

따라서, 전세집을 구할 때는 등기부등본을 확인하고 대리인이 나타난다면 대리인의 신분증을 확인하고 임대인과의 유선 통화를 통해 계약 내용을 다시 한 번 확인하고 전세보증금을 지불할 때는 임대인 명의의 통장에 지불하는 것이 중요합니다. 이를 통해 가짜 임대인을 이용한 전세사기를 예방할 수 있습니다.

'가짜 임대인을 이용한 전세사기는 매년 많은 피해자들을 만들고 있습니다. 따라서 임대인을 확인하는 것은 전세집을 구할 때 가장 중요한 일 중 하나입니다. 하지만, 임대인이 본인이 맞다고 해서 모든 것이 끝나는 것은 아닙니다. 여전히 조심해야할 부분이 있습니다.\n\n먼저, 임대인이 본인이 맞는지 확인하는 것은 등기부등본을 통해 확인할 수 있습니다. 등기부등본은 부동산의 소유자와 관련된 정보를 담고 있는 공적 서류입니다. 등기부등본을 확인하면 임대인의 인적사항과 부동산의 소유자 정보를 확인할 수 있습니다. 하지만, 등기부등본을 위조하는 경우도 있기 때문에 등기부등본을 확인할 때는 본인이 직접 발급받아 확인하는 것이 좋습니다.\n\n또한, 가짜 임대인이 등장하는 경우 대리인이 나타날 수도 있습니다. 대리인이 나타난다면, 대리인의 신분증을 확인하고 대리인이 임대인의 위임을 받았는지 확인해야 합니다. 대리인이 임대인의 위임을 받았다면, 위임장과 인감증명서를 대조해 두 서류에 찍힌 임대인의 인감이 동일한지 체크하는 것이 중요합니다. 또한, 임대인과의 유선 통화를 통해 계약 내용을 다시 한 번 확인하는 것도 좋은 방법입니다.\n\n마지막으로, 전세보증금을 지불할 때는 반드시 임대인 명의의 통장에 지불해야 합니다. 만약 임대인의 요청에 의해 대리인 계좌로 입금해야 하는 경우라면, 특약사항에 임대인의 요청에 따라 대리인 계좌로 입금하게 되었음을 명시해야 합니다. 이를 통해 보증금을 가로채는 사기를 방지할 수 있습니다.\n\n따라서, 전세집을 구할 때는 등기부등본을 확인하고 대리인이 나타난다면 대리인의 신분증을 확인하고 임대인과의 유선 통화를 통해 계약 내용을 다시 한 번 확인하고 전세보증금을 지불할 때는 임대인 명의의 통장에 지불하는 것이 중요합니다. 이를 통해 가짜 임대인을 이용한 전세사기를 예방할 수 있습니다.'

# 3. 계약서 작성할 때

In [13]:
system_template="""
너는 사용자의 전세사기 예방을 돕는 챗봇이야. 너는 여러 챗봇 중 "계약서 작성할 때" 시점의 사용자를 담당해
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [14]:
human_template="""
계약서를 작성할 때 생길 수 있는 전세사기 유형에는 2가지가 있습니다. 아래는 전세사기의 유형과 각 유형에 대한 사례와 예방방법입니다.

1. 월셋집을 전셋집으로 둔갑시킨 경우
사례 : 2019년, 경기도 안산에서는 두 명의 공인중개사가 월셋집을 전셋집으로 둔갑시켜 피해액이 70억원에 달하는 사기를 저질렀어요. 임대인에게는 월세 계약이라고 알리고 세입자와는 전세 계약을 맺은 후, 그 사이에서 전세보증금을 가로챈 거죠.
- 일부 지역에는 나쁜 관행이 있어요. 그건 바로 임대인 없이 임대차계약을 맺는 관행이에요. 공인중개사가 임대인의 대리인을 자처하면서, 임대인이나 대리인 관련 서류를 전혀 준비해주지 않는 거죠. 이 빈틈을 노리고, 사기를 치는 일부 나쁜 공인중개사가 있어요. 예를 들어볼게요. 여기 임대인의 대리인으로서 계약을 하러 나온 공인중개사가 한 명 있어요. 이 공인중개사는 세입자와 보증금 1억 원의 전세계약을 체결해버려요. 하지만 임대인에게는 보증금 1천만 원의 월세 계약을 체결했다고 거짓말을 해요. 이렇게 발생한 총 9천만 원의 차액을 나쁜 공인중개사가 빼돌려요. 공인중개사가 임대인과 세입자 모두를 속이는 거죠.
예방방법 1 - 그렇다면 이러한 불상사를 막기 위해서는 어떻게 해야할까요? 우선, 해당 공인중개업자가 정상적으로 등록된 공인중개업자가 맞는지 사전에 확인해야 해요. 무자격자가 증개업 등록증이나 자격증을 빌려서 중개사무소를 차리는 경우가 생각보다 많아요. 등록된 중개업자인지 여부는 '국가공간정보포털'에서 확인할 수 있어요. 열람공간에 들어가 부동산 중개업조회 메뉴를 클릭하면 지역과 사무소 상호, 공인중개사 이름, 전화번호 등을 입력할 수 있고 이를 통해 등록번호와 중개업자 여부를 확인할 수 있어요.
예방방법 2 - 개업 중인 공인중개사는 공인중개사법 제30조에 따라 보증보험이나 공제상품에 가입해야 하는데요. 이에 따라 공인중개사의 과실이나 고의로 계약자가 금전적 피해를 입었을 때 손해를 보증한다는 내용을 담은 '부동산 공제증서'를 발급하게 돼요. 이 공제증서는 실제 중개사고 때문에 벌어진 손해를 배상해줄 때에 쓰이며, 공인중개사가 사기를 벌이지 않겠 다는 다짐이기도 해요. 그러니 중개업등록증과 공인중개사의 신분증을 확인해 꼭 증서를 받을 수 있도록 해요.
국가공간정보포털 링크 : http://www.nsdi.go.kr/lxportal/?menuno=2679

2. 선순위 임차보증금 및 근저당 허위 고지
사례 : 대구에서는 다가구 주택의 임대인이 다른 세입자들의 임차보증금 규모를 속여 전세계약을 체결하는 사건이 있었어요. 그리고 부산에서는 공동근저당이 설정되어있으나 이를 중요하게 생각하지 않고 전세계약을 체결하였다가 집이 경매에 넘어가 보증금을 회수하지 못하는 경우가 있었어요.
주택 A : 집값 10억/내 보증금 2억/다가구주택A에 살고있던 세입자들의 보증금 즉,선순위 임차보증금 12억
주택 B : 공동 담보 88억/나를 포함한 세입자들의 보증금 70억
주택 A, B중 어느것이 더 안전한 집일까요? - 정답은 둘 다 위험한 집입니다! 다가구주택A에는 너무 많은 보증금들이 이미 나보다 우선하여 묶여 있는 집이기 때문에 위험할 수 있어요. 다세대주택 또는 오피스텔B에는 공동 담보가 너무 커서, 세입자들의 보증금 보호가 제대로 지켜지지 않을 수 있어요. B의 경우, 등기부등본에서 공동담보에 대한 언급을 확인할 수 있어요. 하지만 다가구주택A에 사는 세입자들의 보증금은 등기부등본에 나타나지 않아요. 그래서 좀더 유의해야 해요.
예방방법 1 - 다가구주택의 경우 계약을 하기 전에는 다른 세입자의 보증금 규모를 알기 어려워요. 따라서 반드시 임대인의 동의를 받아 계약 전에 부채 규모를 확인해야 해요. 만일 임대인이 거짓말을 할 경우를 대비하여 특약사항을 기재하는 것을 추천해요.
예방방법 2 - 계약 전에는 임대인의 동의서, 임대인의 신분증 사본, 인감증명서 등 필요서류를 챙겨 확정일자 부여 현황 서류를 발급받도록 해요. 계약 후에는 임대차 계약서와 본인 신분증 사본 등 필요서류를 챙겨 확정일자 부여 현황과 전입세대 열람 내역 2가지 모두 발급해보세요. 확정일자 부여 현황과 전입세대 열람 내역에 기재된 보증금 금액을 통해 이 집에 대해 설정된 부채의 규모를 계산해 볼 수 있어요.

규칙 :
1. 질문이 특정 사기 유형에 해당하지 않는다면 먼저 전세사기 유형 2가지에 대해 답변에서 설명한다.
2. 해당하는 유형별 예방방법을 반드시 답변에 포함한다.

위 내용과 규칙을 참고해서 다음 Question에 1200단어 정도로 답변해.
Question :  {text}
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [15]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [16]:
# prompt template가 2146이고, 사용자의 질문을 고려해서 1500으로 산정
# llm = ChatOpenAI(temperature=0, max_tokens=1500)
llm = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0, max_tokens=1500)
before_contract_chain = LLMChain(
    llm=llm,
    prompt=chat_prompt
)

In [17]:
before_contract_chain.run("전세집을 구하려고 하는데 지금 나는 계약서를 작성하는 중이야 조심해야할 부분은 없을까?")

전세집을 구하려는 경우, 계약서 작성 시에도 전세사기에 대한 예방이 필요합니다. 전세사기 유형에는 월셋집을 전셋집으로 둔갑시키는 경우와 선순위 임차보증금 및 근저당 허위 고지가 있습니다. 이에 대한 예방방법을 알아보겠습니다.

먼저, 월셋집을 전셋집으로 둔갑시키는 경우를 예방하기 위해서는 공인중개사의 등록 여부를 사전에 확인해야 합니다. 등록된 중개업자인지 여부는 국가공간정보포털에서 확인할 수 있습니다. 또한, 개업 중인 공인중개사는 공인중개사법 제30조에 따라 보증보험이나 공제상품에 가입해야 합니다. 이에 따라 공인중개사의 과실이나 고의로 계약자가 금전적 피해를 입었을 때 손해를 보증하는 '부동산 공제증서'를 발급받을 수 있습니다. 따라서 중개업등록증과 공인중개사의 신분증을 확인하고 증서를 받을 수 있도록 해야 합니다.

또한, 선순위 임차보증금 및 근저당 허위 고지에 대한 예방방법은 계약 전에 임대인의 동의를 받아 부채 규모를 확인하는 것입니다. 다가구주택의 경우 계약을 하기 전에는 다른 세입자의 보증금 규모를 알기 어려우므로 반드시 임대인의 동의를 받아 계약 전에 부채 규모를 확인해야 합니다. 만일 임대인이 거짓말을 할 경우를 대비하여 특약사항을 기재하는 것을 추천합니다. 계약 전에는 임대인의 동의서, 임대인의 신분증 사본, 인감증명서 등 필요서류를 챙겨 확정일자 부여 현황 서류를 발급받도록 하고, 계약 후에는 임대차 계약서와 본인 신분증 사본 등 필요서류를 챙겨 확정일자 부여 현황과 전입세대 열람 내역을 발급받아 보증금 금액을 통해 이 집에 대해 설정된 부채의 규모를 계산해 볼 수 있습니다.

따라서, 전세집을 구하려는 경우 계약서 작성 시에도 전세사기에 대한 예방이 필요하며, 위에서 언급한 예방방법을 참고하여 계약서 작성 시에도 주의해야 합니다. 또한, 계약서 작성 전에는 계약서 내용을 꼼꼼히 확인하고, 필요한 서류를 챙겨 확정일자 부여 현황과 전입세대 열람 내역을 발급받아 보증금 금액을 통해 이 집에 대해 설정된 부채의 규모를 계산해 볼 필요가 있습

"전세집을 구하려는 경우, 계약서 작성 시에도 전세사기에 대한 예방이 필요합니다. 전세사기 유형에는 월셋집을 전셋집으로 둔갑시키는 경우와 선순위 임차보증금 및 근저당 허위 고지가 있습니다. 이에 대한 예방방법을 알아보겠습니다.\n\n먼저, 월셋집을 전셋집으로 둔갑시키는 경우를 예방하기 위해서는 공인중개사의 등록 여부를 사전에 확인해야 합니다. 등록된 중개업자인지 여부는 국가공간정보포털에서 확인할 수 있습니다. 또한, 개업 중인 공인중개사는 공인중개사법 제30조에 따라 보증보험이나 공제상품에 가입해야 합니다. 이에 따라 공인중개사의 과실이나 고의로 계약자가 금전적 피해를 입었을 때 손해를 보증하는 '부동산 공제증서'를 발급받을 수 있습니다. 따라서 중개업등록증과 공인중개사의 신분증을 확인하고 증서를 받을 수 있도록 해야 합니다.\n\n또한, 선순위 임차보증금 및 근저당 허위 고지에 대한 예방방법은 계약 전에 임대인의 동의를 받아 부채 규모를 확인하는 것입니다. 다가구주택의 경우 계약을 하기 전에는 다른 세입자의 보증금 규모를 알기 어려우므로 반드시 임대인의 동의를 받아 계약 전에 부채 규모를 확인해야 합니다. 만일 임대인이 거짓말을 할 경우를 대비하여 특약사항을 기재하는 것을 추천합니다. 계약 전에는 임대인의 동의서, 임대인의 신분증 사본, 인감증명서 등 필요서류를 챙겨 확정일자 부여 현황 서류를 발급받도록 하고, 계약 후에는 임대차 계약서와 본인 신분증 사본 등 필요서류를 챙겨 확정일자 부여 현황과 전입세대 열람 내역을 발급받아 보증금 금액을 통해 이 집에 대해 설정된 부채의 규모를 계산해 볼 수 있습니다.\n\n따라서, 전세집을 구하려는 경우 계약서 작성 시에도 전세사기에 대한 예방이 필요하며, 위에서 언급한 예방방법을 참고하여 계약서 작성 시에도 주의해야 합니다. 또한, 계약서 작성 전에는 계약서 내용을 꼼꼼히 확인하고, 필요한 서류를 챙겨 확정일자 부여 현황과 전입세대 열람 내역을 발급받아 보증금 금액을 통해 이 집에 대해 설정된 부채의 규모를 계산해 볼

# 4. 계약한 직후

In [18]:
system_template="""
너는 사용자의 전세사기 예방을 돕는 챗봇이야. 너는 여러 챗봇 중 "계약한 직후" 시점의 사용자를 담당해
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [19]:
human_template="""
계약한 직후 생길 수 있는 전세사기 유형에는 3가지가 있습니다. 아래는 전세사기의 유형과 각 유형에 대한 사례와 예방방법입니다.

1. 전세계약 당일 임대인 변경 및 주택담보대출 실행
사례 : 2021년 4월, A씨는 서울 동작구 상도동의 한 주택에 입주하게 되었어요. 근저당이 없어 권리관계가 깨끗한 집이었죠. A씨는 전입신고와 동시에 확정일자까지 받았 어요. 이쯤 되면 세입자로서 할 수 있는 모든 일을 다 했다고 볼 수 있지요. 그런데 어느 날, 법원으로부터 고지서 한 통을 받게 되었어요. 집이 경매에 넘어가게 되었으니 퇴거해달라는 내용이었죠. 어떻게 된 일일까요? 알고 보니 전입신고 당일, 임대인 B씨가 자산이 거의 없다시피 한 C씨에게 소유권을 넘겼고, C씨는 주택담보대출을 받았어요. 그러다 대출을 갚을 여력이 되지 않았던 C씨는 잠적했고, 집은 경매에 넘어가게 된 거죠.
예방 방법 1 - 전세계약 당일 잔금을 치르기 전에 등기부등본을 확인, 만일 임대인을 변경하는 내용의 등기가 접수되었다면 ‘신청사건 처리 중’으로 명시되기 때문이죠. 이 경우 잔금을 지급하지 말고 어떤 등기가 접수되어 처리중인지 확인해야만 해요. 등기부등본은 이사 당일 뿐 아니라 2-3일 뒤에도 살펴보는 것이 좋아요.
예방 방법 2 - 특약사항을 넣어요, 계약서를 쓸 때, 계약 체결일 다음날까지 소유권 변경, 근저당 설정 등의 행위를 일체 하지 않기로 하는 특약사항을 넣어야 해요.
예방 방법 3 - 불안하다면 전세권을 설정하는 방법도 있어요, 전세권이란, 타인의 부동산을 점유하여 그 부동산의 용도에 따라 사용 및 수익을 얻을 수 있는 권리예요. 쉽게 말해 기존의 임대차계약이 집만 빌리는 것이라고 한다면 전세권 설정은 집과 함께 그 집에 대한 권리관계까지 ‘내’가 차지한다고 볼 수 있어요.

2. 이중계약
사례 : 한 청년 A는 마음에 드는 전세 물건을 찾아 권리관계를 확인하고 임대인을 직접 만나 계약을 체결하였어요. 그러나 이사 당일, A는 자기 집에 다른 세입자 B가 있다는 사실을 알게 되었어요. 임대인이 하나의 주택을 대상으로 2명에게 보증금을 받아 가로챈 것이죠. 이처럼, 임대인이 새로운 임대인에게 보증금을 받은 후 기존 세입자에게 반환하지 않고 잠적하는 사례가 종종 있다고 해요. 하나의 주택을 대상으로 두 사람 이상과 임대차계약을 체결하는 임대인의 목적은 주로 액수가 큰 보증금을 두 배, 세 배로 받는 거죠. 이러한 행위는 그 자체로 불법이므로 이로 인해 누군가 금전적 손해를 입게 된다면 해당 임대인은 배임죄나 사기죄의 명목으로 형사 소송까지 갈 수 있어요.
예방 방법 1 - 깡통주택을 조심해야 해요, 현재까지의 사기 사례를 살펴보면, 보증금이 주택가격과 비슷할수록 이런 사기가 발생할 가능성이 커요. 그래서 보증금을 비롯해 이미 임대인이 집을 담보로 너무 많은 빚을 지고 있다면, 이 빚들의 총합이 주택가격의 80%를 넘는 깡통주택이라면 좀 더 고민해보세요.
예방 방법 2 - 기존 세입자가 이사를 나갔는지 확인하고 잔금을 지급, 임대인이 기존 세입자의 보증금을 돌려줘야 그 사람이 이사를 나갈 텐데, 만약 기존 세입자의 보증금도 제때 돌려주지 않고, 나한테도 보증금도 받아 간 채 잠적하면 안 되니까요. 만약, 임대인과 기존 세입자 모두가 합의하고 내가 보증금을 기존 세입자에게 직접 입금하는 경우가 생길 수도 있어요. 그럴 때는 계약서의 특약사항에 보증금을 기존 세입자의 계좌로 입금한다는 사실을 정확히 적어두어야 해요.

3. 선순위 근저당, 신탁등기 말소 등 특약조건 불이행
사례 : 이사를 앞둔 A씨는 여러 번 집을 보던 도중 마음에 쏙 드는 집을 발견했어요. 곧바로 등기부등본을 떼어 보았죠. 등기부등본을 자세히 살펴보던 A씨는 그 집이 신탁 등기가 되어있는 집이라는 사실을 알게 되었어요. 신탁 부동산은 위험할 소지가 크다는 말을 들었던 A씨는 계약을 고민했어요. 그러자 임대인은 신탁등기를 말소할 테니 걱정하지 말라며 A씨를 안심시켰어요. 결국, 계약서 특약사항에 임대인이 신탁등기를 말소할 것을 적는다는 조건으로 계약하기로 했어요. 마음을 놓은 A씨는 무사히 이사를 마쳤어요. 그런데! 전입신고를 마치고 등기부등본을 떼어 본 A씨는 깜짝 놀랐어요. 아직 신탁등기가 말소되지 않았던 거예요!
예방 방법 - 특약사항에 임대인의 의무를 명확하게 기재하도록 해야 해요. 그러면 근저당권을 말소하지 않을 시, 전세 계약을 취소할 수 있어요. 필요하다면 임대인이 계약서 특약을 지키지 않아서 세입자에 생긴 손해에 대한 배상도 요구해볼 수 있겠죠. 특약을 쓴 뒤, 무엇보다 중요한 것은 임대인이 근저당권 또는 신탁등기를 말소하는지 두 눈으로 직접 확인하는 일이에요. 예를 들어 임대인이 신탁등기를 말소하겠다는 특약을 썼다고 해도, 어차피 신탁등기에서 임대 권한을 가진 것은 신탁회사이기 때문에 유의해야 해요. 아예 계약서에 보증금을 입금할 신탁회사를 정확히 적어두고, 보증금을 신탁회사로 입금하는 게 더 나을 수 있어요.

규칙 :
1. 질문이 특정 사기 유형에 해당하지 않는다면 먼저 전세사기 유형 3가지에 대해 답변에서 설명한다.
2. 해당하는 유형별 예방방법을 반드시 답변에 포함한다.

위 내용과 규칙을 참고해서 다음 Question에 800단어 정도로 답변해줘.
Question :  {text}
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [20]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [21]:
# prompt template가 2587이고, 사용자의 질문을 고려해서 1200으로 산정
# llm = ChatOpenAI(temperature=0, max_tokens=1200)
llm = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0, max_tokens=1200)
after_contract_chain = LLMChain(
    llm=llm,
    prompt=chat_prompt
)

In [22]:
after_contract_chain.run("전세집을 구하려고 하는데 지금 나는 방금 계약을 끝냈어 조심해야할 부분은 없을까?")

전세집을 구하려는 경우, 계약을 체결한 직후에는 여러 가지 전세사기 유형에 노출될 가능성이 높습니다. 따라서, 계약을 체결한 직후에는 반드시 전세사기 유형에 대한 예방방법을 숙지하고, 계약서에 특약사항을 추가하는 등의 조치를 취해야 합니다.

전세사기 유형은 크게 세 가지로 나눌 수 있습니다. 첫 번째는 전세계약 당일 임대인 변경 및 주택담보대출 실행입니다. 이 경우, 등기부등본을 확인하고, 등기부등본에 임대인 변경 내용이 없는지 확인하는 것이 중요합니다. 또한, 계약서에 특약사항을 추가하여, 계약 체결일 다음날까지 소유권 변경, 근저당 설정 등의 행위를 일체 하지 않기로 하는 내용을 명시하는 것이 좋습니다. 불안하다면 전세권을 설정하는 방법도 있습니다.

두 번째는 이중계약입니다. 이중계약은 임대인이 하나의 주택을 대상으로 2명 이상에게 보증금을 받는 경우를 말합니다. 이 경우, 깡통주택을 조심해야 하며, 보증금을 비롯해 이미 임대인이 집을 담보로 너무 많은 빚을 지고 있다면, 이 빚들의 총합이 주택가격의 80%를 넘는 깡통주택이라면 좀 더 고민해보아야 합니다. 또한, 계약서에 특약사항을 추가하여, 기존 세입자가 이사를 나갔는지 확인하고 잔금을 지급하는 등의 내용을 명시하는 것이 좋습니다.

세 번째는 선순위 근저당, 신탁등기 말소 등 특약조건 불이행입니다. 이 경우, 계약서에 특약사항을 추가하여, 임대인이 근저당권을 말소하지 않을 시, 전세 계약을 취소할 수 있도록 하는 등의 내용을 명시하는 것이 좋습니다. 또한, 임대인이 근저당권 또는 신탁등기를 말소하는지 두 눈으로 직접 확인하는 것이 중요합니다.

따라서, 전세집을 구하려는 경우, 계약을 체결한 직후에는 반드시 위의 예방방법을 숙지하고, 계약서에 특약사항을 추가하는 등의 조치를 취해야 합니다. 또한, 계약서를 작성할 때는 상세한 내용을 기재하고, 계약서를 잘 읽어보는 것이 중요합니다. 이를 통해 전세사기로부터 안전하게 자신을 보호할 수 있습니다.

'전세집을 구하려는 경우, 계약을 체결한 직후에는 여러 가지 전세사기 유형에 노출될 가능성이 높습니다. 따라서, 계약을 체결한 직후에는 반드시 전세사기 유형에 대한 예방방법을 숙지하고, 계약서에 특약사항을 추가하는 등의 조치를 취해야 합니다.\n\n전세사기 유형은 크게 세 가지로 나눌 수 있습니다. 첫 번째는 전세계약 당일 임대인 변경 및 주택담보대출 실행입니다. 이 경우, 등기부등본을 확인하고, 등기부등본에 임대인 변경 내용이 없는지 확인하는 것이 중요합니다. 또한, 계약서에 특약사항을 추가하여, 계약 체결일 다음날까지 소유권 변경, 근저당 설정 등의 행위를 일체 하지 않기로 하는 내용을 명시하는 것이 좋습니다. 불안하다면 전세권을 설정하는 방법도 있습니다.\n\n두 번째는 이중계약입니다. 이중계약은 임대인이 하나의 주택을 대상으로 2명 이상에게 보증금을 받는 경우를 말합니다. 이 경우, 깡통주택을 조심해야 하며, 보증금을 비롯해 이미 임대인이 집을 담보로 너무 많은 빚을 지고 있다면, 이 빚들의 총합이 주택가격의 80%를 넘는 깡통주택이라면 좀 더 고민해보아야 합니다. 또한, 계약서에 특약사항을 추가하여, 기존 세입자가 이사를 나갔는지 확인하고 잔금을 지급하는 등의 내용을 명시하는 것이 좋습니다.\n\n세 번째는 선순위 근저당, 신탁등기 말소 등 특약조건 불이행입니다. 이 경우, 계약서에 특약사항을 추가하여, 임대인이 근저당권을 말소하지 않을 시, 전세 계약을 취소할 수 있도록 하는 등의 내용을 명시하는 것이 좋습니다. 또한, 임대인이 근저당권 또는 신탁등기를 말소하는지 두 눈으로 직접 확인하는 것이 중요합니다.\n\n따라서, 전세집을 구하려는 경우, 계약을 체결한 직후에는 반드시 위의 예방방법을 숙지하고, 계약서에 특약사항을 추가하는 등의 조치를 취해야 합니다. 또한, 계약서를 작성할 때는 상세한 내용을 기재하고, 계약서를 잘 읽어보는 것이 중요합니다. 이를 통해 전세사기로부터 안전하게 자신을 보호할 수 있습니다.'

# 5. 입주한 이후

In [23]:
system_template="""
너는 사용자의 전세사기 예방을 돕는 챗봇이야. 너는 여러 챗봇 중 "입주한 이후" 시점의 사용자를 담당해
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [24]:
human_template="""
입주한 이후 생길 수 있는 전세사기 유형에는 2가지가 있습니다. 아래는 전세사기의 유형과 각 유형에 대한 사례와 예방방법입니다.

1. 미납국세 및 임금채권 우선 변제
사례 : 근저당 금액과 선순위 보증금을 다 계산했음에도 불구하고, 전세금을 돌려받지 못하는 경우가 발생할 수 있어요. 그건 바로 세금을 내지 않는 사람들 때문이지요. 만약 내가 살게 될 집의 임대인이 고액체납자라면, 집이 경매에 넘어갔을 때 보증금 전부를 돌려받지 못할 수도 있어요. 실제로 한국자산관리공사의 자료에 따르면 2016년부터 2020년에 이르기까지, 임대인이 국세를 미납하는 바람에 900명의 가까운 세입자가 총 335억의 보증금을 돌려받지 못했다고 해요. 그런가 하면 사용자가 노동자에게 정당한 임금을 지불하지 않는 경우, 이것이 임금채권이 되어 사용자의 뒤를 따라다니게 되는데요. 이렇게 임금채권이 많은 빚쟁이를 임대인으로서 만나게 되면 그 역시, 보증금을 떼일 우려가 있어요.
예방방법 1 - 미납국세 열람제도를 활용, 미납국세 열람제도를 활용해볼 수 있어요. 임대인 동의가 있다면 확인해볼 수 있죠.
예방방법 2 - 사회보험 완납증명서 또는 납세증명서 확인, 사회보험 완납증명서 또는 납세증명서를 임대인 동의로 확인해볼 수 있어요. 사회보험 미납이 곧 임금 체불은 아니지만, 그 가능성과 위험도를 측정하는 데에는 도움을 줄 거예요. 또한 납세증명서를 통해서 확인해볼 수 있어요.
예방방법 3 - 등기부등본을 확인, 임금을 체불했거나 세금을 체납했을 때 간혹 채권자나 국가가 ‘압류’ 혹은 ‘가압류’와 같은 채권보전조치를 취하는 경우가 있거든요. 따라서 만일 등본을 확인했는데 ‘압류’, ‘가압류’, 강제경매 개시 결정’ 등 임대인의 소유권이 위태로워지는 단어들이 보인다면 반드시 피하세요!
예방방법 4 - 전세보증금반환보증에 가입, 내가 미처 예방할 수 없었던 문제가 생겼을 때, 보증보험을 통해 보증금을 보호받을 수 있어요. 단! 압류, 가압류 등 소유권에 대한 권리침해 상황이 발생한다면 보증보험 가입이 안 돼요. 그러니까 대항력을 갖추며 최대한 빠르게 보증보험에 가입하는 것이 좋아요. 집을 구할 때부터, 전세보증금반환보증에 가입할 수 있는 집을 찾는 것도 좋은 방법이에요. 예를 들어, 여러분이 중소기업취업청년 전월세보증금 대출을 100%로 받으셨다면, 대출 상품 자체가 보증보험에 가입되어 있어서 걱정하지 않으셔도 돼요.

2. 세입자의 전출신고 후 벌어지는 전세사기
사례 : 전세 세입자인 A씨는 어느 날 임대인에게서 전화 한 통을 받았어요. 대출을 받아야 한다고, 잠시 집에서 전출해줄 수 있냐는 임대인의 요청이 있었어요. A씨는 별문제가 없을 거라 판단해 임대인이 대출받는 날 잠깐 전출하고 다시 전입신고를 했죠. 그런데 이후 A씨가 전세자금 대출을 연장하려고 하자 은행에선 A씨의 대출 연장을 거절했어요.
이러한 사기가 벌어지는 이유 1 - 전입신고를 한 후에는 임대인이 무엇을 요구하든 전출신고를 하지 않는 게 좋아요. 전출신고를 하면 그 순간부터 내 대항력이 와장창! 깨지게 되거든요. 내가 이 집의 세입자로 권리를 보장받기 위해서는 대항력을 유지하고 있어야 하는데, 중요한 요건 중 하나인 ‘전입신고’가 해제되는 것이라, 매우 위험해요. 분명 내가 먼저 살고 있었는데, 내가 전출신고한 사이에 들어온 빛이 나보다 우선하게 되는 것이기도 하고요. 이런 경우, 집이 경매에 넘어가면 그 빚을 보전하느라 내 보증금을 돌려받지 못하게 될 수도 있어요.
이러한 사기가 벌어지는 이유 2 - 세입자가 전출신고를 한 사이에 임대인이 집을 필수도 있어요. 이렇게 되면 소유권이 타인에게로 넘어가게 되고, 내 대항력이 깨진 채로 정말 곤란한 일이 벌어질 수 있어요. 새 임대인이 나보고 나가라고 했을 때, 이미 내가 전출신고 후 대항력이 깨진 상태이기 때문에 나의 상황을 보호받는 과정이 몹시 복잡해질 수 있어요.

규칙 :
1. 질문이 특정 사기 유형에 해당하지 않는다면 먼저 전세사기 유형 2가지에 대해 답변에서 설명한다.
2. 해당하는 유형별 예방방법을 반드시 답변에 포함한다.

위 내용과 규칙을 참고해서 다음 Question에 1000단어 정도로 답변해줘.
Question :  {text}
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [25]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [26]:
# prompt template가 2020이고, 사용자의 질문을 고려해서 1500으로 산정
# llm = ChatOpenAI(temperature=0, max_tokens=1500)
llm = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0, max_tokens=1500)
after_home_chain = LLMChain(
    llm=llm,
    prompt=chat_prompt
)

In [27]:
after_home_chain.run("전세집을 구해서 계약을 끝냈고 입주 완료한 상태야, 이런 경우에도 조심할 부분이 남아있을까?")

네, 전세집을 구해서 계약을 끝냈고 입주 완료한 상태라 하더라도 여전히 조심해야 할 부분이 있어요. 입주한 이후에도 전세사기가 발생할 수 있는데, 그 유형은 크게 두 가지로 나눌 수 있어요.

첫 번째 유형은 미납국세 및 임금채권 우선 변제에 대한 것이에요. 이 경우, 사용자가 이미 입주를 완료한 상태에서도 임대인이 국세를 미납하거나 노동자에게 임금을 지불하지 않는 등의 문제가 발생할 수 있어요. 이 경우, 사용자는 미납국세 열람제도를 활용하거나 사회보험 완납증명서 또는 납세증명서를 확인해볼 수 있어요. 또한 등기부등본을 확인하거나 전세보증금반환보증에 가입하는 것도 좋은 방법이에요.

두 번째 유형은 세입자의 전출신고 후 벌어지는 전세사기에 대한 것이에요. 이 경우, 사용자가 이미 입주를 완료한 상태에서도 세입자가 전출신고를 한 후에는 대항력이 깨지게 되어 위험해져요. 따라서 사용자는 전입신고를 한 후에는 전출신고를 하지 않는 것이 좋아요. 또한, 세입자가 전출신고를 한 사이에 임대인이 집을 필수로 할 경우, 소유권이 타인에게로 넘어가게 되어 사용자의 대항력이 깨지게 되므로, 이 경우에는 사용자가 보호받을 수 있는 방법을 찾아보는 것이 좋아요.

따라서, 전세집을 구해서 계약을 끝냈고 입주 완료한 상태라 하더라도, 사용자는 여전히 조심해야 해요. 미납국세 및 임금채권 우선 변제와 세입자의 전출신고 후 벌어지는 전세사기에 대한 예방방법을 알고, 가능한 모든 대처책을 마련해두는 것이 좋아요. 이렇게 하면 사용자가 전세사기에 대한 위험을 최소화할 수 있을 거예요.

'네, 전세집을 구해서 계약을 끝냈고 입주 완료한 상태라 하더라도 여전히 조심해야 할 부분이 있어요. 입주한 이후에도 전세사기가 발생할 수 있는데, 그 유형은 크게 두 가지로 나눌 수 있어요.\n\n첫 번째 유형은 미납국세 및 임금채권 우선 변제에 대한 것이에요. 이 경우, 사용자가 이미 입주를 완료한 상태에서도 임대인이 국세를 미납하거나 노동자에게 임금을 지불하지 않는 등의 문제가 발생할 수 있어요. 이 경우, 사용자는 미납국세 열람제도를 활용하거나 사회보험 완납증명서 또는 납세증명서를 확인해볼 수 있어요. 또한 등기부등본을 확인하거나 전세보증금반환보증에 가입하는 것도 좋은 방법이에요.\n\n두 번째 유형은 세입자의 전출신고 후 벌어지는 전세사기에 대한 것이에요. 이 경우, 사용자가 이미 입주를 완료한 상태에서도 세입자가 전출신고를 한 후에는 대항력이 깨지게 되어 위험해져요. 따라서 사용자는 전입신고를 한 후에는 전출신고를 하지 않는 것이 좋아요. 또한, 세입자가 전출신고를 한 사이에 임대인이 집을 필수로 할 경우, 소유권이 타인에게로 넘어가게 되어 사용자의 대항력이 깨지게 되므로, 이 경우에는 사용자가 보호받을 수 있는 방법을 찾아보는 것이 좋아요.\n\n따라서, 전세집을 구해서 계약을 끝냈고 입주 완료한 상태라 하더라도, 사용자는 여전히 조심해야 해요. 미납국세 및 임금채권 우선 변제와 세입자의 전출신고 후 벌어지는 전세사기에 대한 예방방법을 알고, 가능한 모든 대처책을 마련해두는 것이 좋아요. 이렇게 하면 사용자가 전세사기에 대한 위험을 최소화할 수 있을 거예요.'

# 6. 계약기간이 끝난 후

In [28]:
system_template="""
너는 사용자의 전세사기 예방을 돕는 챗봇이야. 너는 여러 챗봇 중 "계약기간이 끝난 후" 시점의 사용자를 담당해.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [29]:
human_template="""
계약기간이 끝난 후 생길 수 있는 전세사기 유형에는 계약이 끝났는데 임대인이 보증금을 안돌려주는 경우가 있을 수 있습니다.
대처방법 1 - 내용증명을 보내기, ‘내용증명’이란 쉽게 말해 우체국에서 공적 증명을 해주는 거예요. 내가 누군가에게 어떤 서류를 전달할 때 내용증명으로 보내면, 우체국에서는 그 서류가 전달되었는지뿐만 아니라 그 내용까지도 증명해주는 거죠. 간단하지만 생각보다 받는 상대방은 심리적 압박을 받을 수 있어요. 물론 실제 법적인 공방이 오가게 될 때 증거로서 작용할 수도 있죠.
대처방법 2 - '임차권등기명령'을 신청하기, 보증금을 반환받지 못한 상황에서 급하게 다음 집으로 이사를 해야 하는 상황을 겪고 있는 분들이 많을 거예요. 이런 경우, 내가 이사를 하더라도 임차권등기명령을 완료해 놓으면 대항력을 유지할 수가 있어요. 쉽게 말해 임차권에 대한 권리와 보증금에 대한 권리가 보호받게 되는 거죠. 더군다나 임차권등기명령은 임대인에게 매우 치명적이라고 해요. 등기부등본 을구에 ‘주택임차권’이라는 이름으로 그대로 적히게 되거든요. 그리고 누가 들어오든 이미 ‘나’의 보증금에 대한 우선변제권이 설정되어 있어서 임대인이 마음대로 다음 세입자를 입주시키고 나를 무시할 수 없어요. 임차권이 등기부등본에 쓰이면 다음 세입자보다 일단 내 보증금이 우선하여 보호받게 되거든요. 반대로, 내가 알아보고 있는 집의 등기부등본에 임차권등기명령이 표시되어 있다면? 이미 그 집에 살던 세입자가 여전히 보증금을 돌려받고 있지 못한다는 뜻이니, 계약을 더 신중히 고민해주세요.
대처방법 3 - 지급명령 신청 활용하기, 이 절차는 전세금 반환 소송으로 들어가기 전 최후의 수단이에요. 전세금 반환 소송에 비해 상대적으로 소요 기간이 짧아서 지급 결정 확정까지 한 달 안에 가능해요. 소요 비용 역시 적기 때문에 경제적으로도 유리해요.

규칙 :
1. 질문이 특정 사기 유형에 해당하지 않는다면 먼저 임대인이 보증금을 돌려주지 않는 경우에 대한 대처방법을 답변한다.

위 내용과 규칙을 참고해서 다음 Question에 1500단어 정도로 답변해줘.
Question :  {text}
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [30]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [31]:
# prompt template가 1025이고, 사용자의 질문을 고려해서 2500으로 산정
# llm = ChatOpenAI(temperature=0, max_tokens=1500)
llm = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0, max_tokens=2500)
after_contract_period_chain = LLMChain(
    llm=llm,
    prompt=chat_prompt
)

In [32]:
after_contract_period_chain.run("계약기간이 끝났는데 집주인이 보증금을 돌려주지 않아 어떻게 해야할까?")

계약기간이 끝났는데 집주인이 보증금을 돌려주지 않는 경우, 먼저 내용증명을 보내는 것이 좋습니다. 내용증명은 우체국에서 공적 증명을 해주는 것으로, 내가 누군가에게 어떤 서류를 전달할 때 내용증명으로 보내면, 우체국에서는 그 서류가 전달되었는지뿐만 아니라 그 내용까지도 증명해줍니다. 이를 통해 상대방에게 심리적 압박을 줄 수 있습니다. 또한, 실제 법적인 공방이 오가게 될 때 증거로서 작용할 수도 있습니다.

만약 내용증명으로도 보증금을 돌려받지 못한다면, 임차권등기명령을 신청하는 것이 좋습니다. 임차권등기명령은 임대인에게 매우 치명적이며, 등기부등본에 ‘주택임차권’이라는 이름으로 그대로 적히게 됩니다. 이를 통해 임차권에 대한 권리와 보증금에 대한 권리가 보호받게 되며, 임차권이 등기부등본에 쓰이면 다음 세입자보다 일단 내 보증금이 우선하여 보호받게 됩니다. 반대로, 내가 알아보고 있는 집의 등기부등본에 임차권등기명령이 표시되어 있다면 이미 그 집에 살던 세입자가 여전히 보증금을 돌려받고 있지 못한다는 뜻이니, 계약을 더 신중히 고민해야 합니다.

마지막으로, 지급명령 신청을 활용하는 것도 좋은 방법입니다. 이 절차는 전세금 반환 소송으로 들어가기 전 최후의 수단이며, 전세금 반환 소송에 비해 상대적으로 소요 기간이 짧아서 지급 결정 확정까지 한 달 안에 가능합니다. 소요 비용 역시 적기 때문에 경제적으로도 유리합니다.

따라서, 계약기간이 끝났는데 집주인이 보증금을 돌려주지 않는 경우에는 먼저 내용증명을 보내고, 그래도 보증금을 돌려받지 못한다면 임차권등기명령을 신청하거나 지급명령 신청을 활용하는 것이 좋습니다. 하지만, 이러한 절차를 거치더라도 상대방이 돈을 돌려주지 않을 수도 있으므로, 계약을 체결할 때는 신중히 고민하고, 계약서에 보증금 반환 조항을 명확하게 기재하는 것이 중요합니다.

'계약기간이 끝났는데 집주인이 보증금을 돌려주지 않는 경우, 먼저 내용증명을 보내는 것이 좋습니다. 내용증명은 우체국에서 공적 증명을 해주는 것으로, 내가 누군가에게 어떤 서류를 전달할 때 내용증명으로 보내면, 우체국에서는 그 서류가 전달되었는지뿐만 아니라 그 내용까지도 증명해줍니다. 이를 통해 상대방에게 심리적 압박을 줄 수 있습니다. 또한, 실제 법적인 공방이 오가게 될 때 증거로서 작용할 수도 있습니다.\n\n만약 내용증명으로도 보증금을 돌려받지 못한다면, 임차권등기명령을 신청하는 것이 좋습니다. 임차권등기명령은 임대인에게 매우 치명적이며, 등기부등본에 ‘주택임차권’이라는 이름으로 그대로 적히게 됩니다. 이를 통해 임차권에 대한 권리와 보증금에 대한 권리가 보호받게 되며, 임차권이 등기부등본에 쓰이면 다음 세입자보다 일단 내 보증금이 우선하여 보호받게 됩니다. 반대로, 내가 알아보고 있는 집의 등기부등본에 임차권등기명령이 표시되어 있다면 이미 그 집에 살던 세입자가 여전히 보증금을 돌려받고 있지 못한다는 뜻이니, 계약을 더 신중히 고민해야 합니다.\n\n마지막으로, 지급명령 신청을 활용하는 것도 좋은 방법입니다. 이 절차는 전세금 반환 소송으로 들어가기 전 최후의 수단이며, 전세금 반환 소송에 비해 상대적으로 소요 기간이 짧아서 지급 결정 확정까지 한 달 안에 가능합니다. 소요 비용 역시 적기 때문에 경제적으로도 유리합니다.\n\n따라서, 계약기간이 끝났는데 집주인이 보증금을 돌려주지 않는 경우에는 먼저 내용증명을 보내고, 그래도 보증금을 돌려받지 못한다면 임차권등기명령을 신청하거나 지급명령 신청을 활용하는 것이 좋습니다. 하지만, 이러한 절차를 거치더라도 상대방이 돈을 돌려주지 않을 수도 있으므로, 계약을 체결할 때는 신중히 고민하고, 계약서에 보증금 반환 조항을 명확하게 기재하는 것이 중요합니다.'

# 7. 명의도용 대출 사기, 브로커를 통한 전세보증 사기

In [33]:
system_template="""
너는 사용자의 전세사기 예방을 돕는 챗봇이야. 너는 여러 챗봇 중 "명의도용 대출사기, 브로커를 통한 전세보증 사기"에 대한 내용을 답변하는 챗봇이야.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [34]:
human_template="""
1. 명의도용 대출 사기
사례 : 지난 2016년, 명의도용으로 대출금을 가로챈 사기꾼 최모 씨의 범행이 드러났어요. 최 씨는 급전이 필요하지만 일반 대출이 어려운 사람들에게 접근해, “가짜 전세계약서로 금리가 낮은 전세자금 대출을 받아주겠다”고 했어요. 정부 지원 전세대출 상품은 전세계약서를 내고 대출 신청만 하면 신용등급이 낮아도 무담보 전세대출을 받을 수 있다는 사실을 악용한 거죠. 이런 수법으로 최 씨는 은행 18곳에서 모두 대출 승인을 받았고 대출금 4억5천만 원을 전부 가로챘어요. 이런 사례를 명의도용 대출이라고 해요. 실제 전세계약이 체결되지 않았는데 임대인과 세입자의 명의를 도용해 가짜 전세계약서를 작성해 여러 군데의 금융기관에서 대출을 실행하는 거죠. 이 밖에도 임대인이 세입자에게 잠시만 명의를 빌려달라고 해서 몰래 대출을 받으려고 하는 등의 명의도용 사기는 빈번하게 일어난답니다. 이 경우에 아무리 범죄행위를 몰랐다고 해도 명의 제공자에게 책임이 전가될 수 있으니 주의해야 해요.
예방 방법 1 - 신분증, 인감 빌려주지 않기
예방 방법 2 - 계약서를 실제 계약 내용과 다르게 작성하지 않기

2. 브로커를 통한 전세보증 사기
사례 : 전세금안심대출보증을 악용해 주택도시보증공사에게 손실을 떠넘기는 사기 사건들이 벌어지고 있다고 해요. 전세보증금을 반환할 형편이 안 되는 사람에게 소유권을 넘기고 세입자는 전세금안심대출보증에 가입해 최대한도로 전세대출을 실행하는 거죠. 전세계약이 종료되면 임대인은 파산 신청을 하는 등의 방식으로 보증금을 반환하지 않고, 세입자는 주택도시보증공사에게 보증금을 돌려달라고 하게 되고요. 특히, 브로커가 명의를 빌려주면 대가를 지급하겠다고 꼬드겨 세입자이나 임대인의 명의를 도용하는 사례들이 있다고 하니 조심하세요! 이런 조직적인 사기 사건에 가담할 경우, 부동산 실명법 위반이나 사기죄 등으로 처벌받을 수 있으니 각별한 주의가 필요해요. 이렇게 되면, 나도 전세사기에 가담하고 휘말리게 되는 것이기 때문에, 처음부터 이런 제안에 응하지 않는 것이 중요해요. 브로커나 임대인이 실제 계약하고자 하는 내용과 다른 계약 방식을 제안했을 때 수락 하지 않아야 해요. 실제와 다른 이면계약을 작성하면 전세금안심대출보증에 가입했더라도 공사의 보증 의무가 없어서 공사로부터도 전세금을 돌려받지 못할 수 있어요.
예방 방법 1 - 신분증, 인감 빌려주지 않기
예방 방법 2 - 적법한 계약 절차가 아니면 거절하기

규칙 :
1. 질문이 특정 사기 유형에 해당하지 않는다면 먼저 전세사기 유형 2가지에 대해 답변에서 설명한다.
2. 해당하는 유형별 예방방법을 반드시 답변에 포함한다.

위 내용과 규칙을 참고해서 다음 Question에 1000단어 정도로 답변해줘.
Question :  {text}
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [35]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [36]:
# prompt template가 1323이고, 사용자의 질문을 고려해서 2500으로 산정
# llm = ChatOpenAI(temperature=0, max_tokens=2500)
llm = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0, max_tokens=2500)
broker_chain = LLMChain(
    llm=llm,
    prompt=chat_prompt
)

In [37]:
broker_chain.run("요즘 명의도용 대출 사기가 많다는데 어떻게 예방해야할까?")

요즘 명의도용 대출 사기가 많이 일어나고 있어서, 이에 대한 예방 방법을 알아보겠습니다. 명의도용 대출 사기는 임대인과 세입자의 명의를 도용해 가짜 전세계약서를 작성해 여러 군데의 금융기관에서 대출을 실행하는 것입니다. 이 경우에는 실제 전세계약이 체결되지 않았는데 대출을 받는 것이기 때문에, 대출금을 상환하지 못하면 명의 제공자에게 책임이 전가될 수 있습니다. 따라서, 명의도용 대출 사기를 예방하기 위해서는 다음과 같은 방법을 고려해볼 수 있습니다.

첫째, 신분증, 인감을 빌려주지 않기
명의도용 대출 사기의 경우, 임대인이나 세입자의 신분증이나 인감을 빌려주면서 대출을 실행하는 경우가 많습니다. 따라서, 신분증이나 인감을 빌려주지 않는 것이 중요합니다. 또한, 신분증이나 인감을 빌려주기 전에 상대방의 신분을 확인하는 것도 중요합니다. 만약 상대방이 은행이나 금융기관에서 대출을 받으려고 한다면, 해당 기관에 전화를 걸어 확인하는 것이 좋습니다.

둘째, 계약서를 실제 계약 내용과 다르게 작성하지 않기
명의도용 대출 사기의 경우, 실제 계약 내용과 다르게 작성된 계약서를 이용해 대출을 실행하는 경우가 많습니다. 따라서, 계약서를 작성할 때는 실제 계약 내용과 다르게 작성하지 않도록 주의해야 합니다. 또한, 계약서를 작성할 때는 상대방의 신분증을 확인하고, 계약 내용을 상세히 확인하는 것이 좋습니다.

따라서, 명의도용 대출 사기를 예방하기 위해서는 상대방의 신분을 확인하고, 신분증이나 인감을 빌려주지 않는 것이 중요합니다. 또한, 계약서를 작성할 때는 실제 계약 내용과 다르게 작성하지 않도록 주의해야 합니다. 이러한 예방 방법을 지켜서 명의도용 대출 사기를 예방할 수 있습니다.

'요즘 명의도용 대출 사기가 많이 일어나고 있어서, 이에 대한 예방 방법을 알아보겠습니다. 명의도용 대출 사기는 임대인과 세입자의 명의를 도용해 가짜 전세계약서를 작성해 여러 군데의 금융기관에서 대출을 실행하는 것입니다. 이 경우에는 실제 전세계약이 체결되지 않았는데 대출을 받는 것이기 때문에, 대출금을 상환하지 못하면 명의 제공자에게 책임이 전가될 수 있습니다. 따라서, 명의도용 대출 사기를 예방하기 위해서는 다음과 같은 방법을 고려해볼 수 있습니다.\n\n첫째, 신분증, 인감을 빌려주지 않기\n명의도용 대출 사기의 경우, 임대인이나 세입자의 신분증이나 인감을 빌려주면서 대출을 실행하는 경우가 많습니다. 따라서, 신분증이나 인감을 빌려주지 않는 것이 중요합니다. 또한, 신분증이나 인감을 빌려주기 전에 상대방의 신분을 확인하는 것도 중요합니다. 만약 상대방이 은행이나 금융기관에서 대출을 받으려고 한다면, 해당 기관에 전화를 걸어 확인하는 것이 좋습니다.\n\n둘째, 계약서를 실제 계약 내용과 다르게 작성하지 않기\n명의도용 대출 사기의 경우, 실제 계약 내용과 다르게 작성된 계약서를 이용해 대출을 실행하는 경우가 많습니다. 따라서, 계약서를 작성할 때는 실제 계약 내용과 다르게 작성하지 않도록 주의해야 합니다. 또한, 계약서를 작성할 때는 상대방의 신분증을 확인하고, 계약 내용을 상세히 확인하는 것이 좋습니다.\n\n따라서, 명의도용 대출 사기를 예방하기 위해서는 상대방의 신분을 확인하고, 신분증이나 인감을 빌려주지 않는 것이 중요합니다. 또한, 계약서를 작성할 때는 실제 계약 내용과 다르게 작성하지 않도록 주의해야 합니다. 이러한 예방 방법을 지켜서 명의도용 대출 사기를 예방할 수 있습니다.'

# 8. 추가 정보를 요청하는 체인

In [38]:
from langchain.chains import LLMChain

default_message = """
해당 질문에는 질문자님의 계약 시점에대한 정보가 부족합니다.

계약 시점 예시 :
1. 집 고를 때
2. 임대인 확인할 때
3. 계약서 작성할 때
4. 계약한 직후
5. 입주한 이후
6. 계약기간이 끝난 후

위와 같은 정확한 시점을 명시해주시면 더욱 상세한 답변이 가능합니다.
"""

class FakeLLMChain(LLMChain):
    def run(self, inputs):
        return default_message

fake_llm_chain = FakeLLMChain(llm=llm, prompt=chat_prompt)
output = fake_llm_chain.run("hi")
print(output)


해당 질문에는 질문자님의 계약 시점에대한 정보가 부족합니다.

계약 시점 예시 :
1. 집 고를 때
2. 임대인 확인할 때
3. 계약서 작성할 때
4. 계약한 직후
5. 입주한 이후
6. 계약기간이 끝난 후

위와 같은 정확한 시점을 명시해주시면 더욱 상세한 답변이 가능합니다.



# 이러한 Chain들을 Tools로 사용하는 Agent 생성

In [72]:
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType

from langchain.memory import ConversationBufferWindowMemory

In [77]:
tools = [
    Tool(
        name = "1.'집 고를때' 시점에 발생하는 전세사기에 대한 답변을 하는 챗봇",
        func=select_home_chain.run,
        description="집을 고르는 시점에 발생하는 전세사기에 대한 답을 해야할 때 유용합니다.",
        return_direct=True,
    ),
    Tool(
        name = "2.'임대인 확인할 때' 시점에 발생하는 전세사기에 대한 답변을 하는 챗봇",
        func=check_lessor_chain.run,
        description="임대인을 확인하는 시점에 발생하는 전세사기에 대한 답을 해야할 때 유용합니다.",
        return_direct=True,
        
    ),
    Tool(
        name = "3.'계약서 작성할 때' 시점에 발생하는 전세사기에 대한 답변을 하는 챗봇",
        func=before_contract_chain.run,
        description="계약서를 작성하는 시점에 발생하는 전세사기에 대한 답을 해야할 때 유용합니다.",
        return_direct=True,
        
    ),
    Tool(
        name = "4.'계약한 직후' 시점에 발생하는 전세사기에 대한 답변을 하는 챗봇",
        func=after_contract_chain.run,
        description="계약한 직후 시점에 발생하는 전세사기에 대한 답을 해야할 때 유용합니다.",
        return_direct=True,
        
    ),
    Tool(
        name = "5.'입주한 이후' 시점에 발생하는 전세사기에 대한 답변을 하는 챗봇",
        func=after_home_chain.run,
        description="입주한 이후 시점에 발생하는 전세사기에 대한 답을 해야할 때 유용합니다.",
        return_direct=True,
        
    ),
    Tool(
        name = "6.'계약기간이 끝난 후' 시점에 발생하는 전세사기에 대한 답변을 하는 챗봇",
        func=after_contract_period_chain.run,
        description="계약기간이 끝난 후 시점에 발생하는 전세사기에 대한 답을 해야할 때 유용합니다.",
        return_direct=True,
        
    ),
    Tool(
        name = "7.'명의도용 대출 사기', '브로커를 통한 전세보증 사기'에 대한 답변을 하는 챗봇",
        func=broker_chain.run,
        description="'명의도용 대출 사기', '브로커를 통한 전세보증 사기'에 대한 답을 해야할 때 유용합니다.",
        return_direct=True,
        
    ),
    Tool(
        name = "추가정보를 요청하는 챗봇",
        func=fake_llm_chain.run,
        description="질문에 특정 시점에대한 명시가 되어있지 않은 경우 이 도구를 사용합니다.",
        return_direct=True,
    
    ),
]

In [78]:
memory = ConversationBufferWindowMemory(k=1, memory_key="chat_history", return_messages=True)

In [79]:
agent_chain = initialize_agent(tools, ChatOpenAI(temperature=0, max_tokens=1000), agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

In [80]:
agent_chain.run("전세사기의 유형에 대해 알려줘")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "전세사기의 유형은 다양합니다. 대표적으로는 '보증금 사기', '계약금 사기', '중개 수수료 사기', '임대료 사기', '명의도용 대출 사기', '브로커를 통한 전세보증 사기' 등이 있습니다."
}

> Finished chain.


"전세사기의 유형은 다양합니다. 대표적으로는 '보증금 사기', '계약금 사기', '중개 수수료 사기', '임대료 사기', '명의도용 대출 사기', '브로커를 통한 전세보증 사기' 등이 있습니다."

In [81]:
agent_chain.run("나는 현재 전세집을 고르고 있는 시점이야, 내가 유의해야할 점에는 어떤 것이 있을까?")



> Entering new AgentExecutor chain...
{
    "action": "1.'집 고를때' 시점에 발생하는 전세사기에 대한 답변을 하는 챗봇",
    "action_input": "전세집을 고를 때 유의해야할 점에 대해 알려주세요."
}전세집을 고를 때는 깡통주택에 대한 위험성을 인지하고, 등기부등본을 꼭 확인해야 합니다. 깡통주택은 임대인이 갭투기로 집을 사들인 뒤, 세입자에게 전세로 내놓는 집으로, 보증금과 대출금의 총합이 집값의 80%를 넘는 집을 의미합니다. 이러한 집은 임대인이 빚 상환을 회피하기 위해 경매에 넘어가거나, 계약이 끝나고 이사를 나갈 때 제때 보증금을 돌려주지 못할 가능성이 높습니다. 따라서 등기부등본을 통해 해당 집의 빚 상황을 확인하고, 깡통주택인지 아닌지를 판단해야 합니다.

또한, 등기부등본을 확인할 때는 표제부, 갑구, 을구를 자세히 살펴봐야 합니다. 갑구에서는 소유권에 대한 권리를 확인할 수 있고, 을구에서는 소유권 외 권리를 확인할 수 있습니다. 이를 통해 해당 집에 대한 소유권이나 제한사항이 있는지, 빚이 얼마나 있는지 등을 파악할 수 있습니다.

또한, 전세계약을 체결할 때는 계약서를 꼼꼼히 읽어보고, 모든 조건을 명확히 하고 서면으로 남겨야 합니다. 보증금의 반환일, 반환금액, 계약기간 등을 명확히 하고, 임대인과의 소통을 원활히 해야 합니다. 또한, 계약서에는 임대인의 연락처와 주소, 계약금액, 계약기간 등이 명시되어 있어야 합니다.

마지막으로, 전세계약을
Observation: 전세집을 고를 때는 깡통주택에 대한 위험성을 인지하고, 등기부등본을 꼭 확인해야 합니다. 깡통주택은 임대인이 갭투기로 집을 사들인 뒤, 세입자에게 전세로 내놓는 집으로, 보증금과 대출금의 총합이 집값의 80%를 넘는 집을 의미합니다. 이러한 집은 임대인이 빚 상환을 회피하기 위해 경매에 넘어가거나, 계약이 끝나고 이사를 나갈 때 제때 보증금을 돌려주지 못할 가능성이 높습니다. 따라서 등기부등본을 통해 해당 집의

'전세집을 고를 때는 깡통주택에 대한 위험성을 인지하고, 등기부등본을 꼭 확인해야 합니다. 깡통주택은 임대인이 갭투기로 집을 사들인 뒤, 세입자에게 전세로 내놓는 집으로, 보증금과 대출금의 총합이 집값의 80%를 넘는 집을 의미합니다. 이러한 집은 임대인이 빚 상환을 회피하기 위해 경매에 넘어가거나, 계약이 끝나고 이사를 나갈 때 제때 보증금을 돌려주지 못할 가능성이 높습니다. 따라서 등기부등본을 통해 해당 집의 빚 상황을 확인하고, 깡통주택인지 아닌지를 판단해야 합니다.\n\n또한, 등기부등본을 확인할 때는 표제부, 갑구, 을구를 자세히 살펴봐야 합니다. 갑구에서는 소유권에 대한 권리를 확인할 수 있고, 을구에서는 소유권 외 권리를 확인할 수 있습니다. 이를 통해 해당 집에 대한 소유권이나 제한사항이 있는지, 빚이 얼마나 있는지 등을 파악할 수 있습니다.\n\n또한, 전세계약을 체결할 때는 계약서를 꼼꼼히 읽어보고, 모든 조건을 명확히 하고 서면으로 남겨야 합니다. 보증금의 반환일, 반환금액, 계약기간 등을 명확히 하고, 임대인과의 소통을 원활히 해야 합니다. 또한, 계약서에는 임대인의 연락처와 주소, 계약금액, 계약기간 등이 명시되어 있어야 합니다.\n\n마지막으로, 전세계약을'

In [83]:
agent_chain.run("나는 현재 계약기간이 끝난 시점이야, 내가 유의해야할 전세사기의 유형에는 어떤 것이 있을까?")



> Entering new AgentExecutor chain...
{
    "action": "7.'명의도용 대출 사기', '브로커를 통한 전세보증 사기'에 대한 답변을 하는 챗봇",
    "action_input": ""
}전세보증금을 반환하지 않는 임대인과 세입자의 명의를 도용해 대출을 실행하는 명의도용 대출사기와 브로커를 통한 전세보증 사기에 대해 더 자세히 알고 싶어요. 이런 사기를 당하지 않으려면 어떤 예방 방법이 있을까요?
Observation: 전세보증금을 반환하지 않는 임대인과 세입자의 명의를 도용해 대출을 실행하는 명의도용 대출사기와 브로커를 통한 전세보증 사기에 대해 더 자세히 알고 싶어요. 이런 사기를 당하지 않으려면 어떤 예방 방법이 있을까요?


> Finished chain.


'전세보증금을 반환하지 않는 임대인과 세입자의 명의를 도용해 대출을 실행하는 명의도용 대출사기와 브로커를 통한 전세보증 사기에 대해 더 자세히 알고 싶어요. 이런 사기를 당하지 않으려면 어떤 예방 방법이 있을까요?'